In [283]:
import numpy as np
import cv2
import math

In [284]:
def draw_matches(image1, image2, kp1, kp2, matches):
    img_matches = cv2.drawMatches(image1, kp1, image2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    # Display the matches
    cv2.imshow("Sift_Matches", img_matches)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [285]:
def get_homography_matrix(kp1, kp2, matches):
    """ 
        Takes the matches and the keypoints for two images and computes a homography matrix.
        Written for the purpose of using inside a RANSAC algorithm
    """
    
    h = []

    for ind in range(len(matches)):
        p = kp1[matches[ind].queryIdx].pt
        q = kp2[matches[ind].trainIdx].pt
        x, y = p
        _x, _y = q
        h_part1, h_part2 = [x, y, 1, 0, 0, 0, -_x*x, -_x*y, -_x], [0, 0, 0, x, y, 1, -_y*x, -_y*y, -_y]
        h.append(h_part1)
        h.append(h_part2)

    _, _, V = np.linalg.svd(h)
    h = V[-1].reshape(3, 3)
    
    return h

In [286]:
def check_ransac_error(kp1, kp2, matches, h, threshold):

    # inline functions for estimating points in image 2 from image 1 using the homography matrix
    x_prime = lambda x, y, h1: (h1[0][0] * x + h1[0][1] * y + h1[0][2]) / (h1[2][0] * x + h1[2][1] * y + h1[2][2])
    y_prime = lambda x, y, h1: (h1[1][0] * x + h1[1][1] * y + h1[1][2]) / (h1[2][0] * x + h1[2][1] * y + h1[2][2])
    
    inliers = 0
    for ind in range(len(matches)):
        x1, y1 = kp1[matches[ind].queryIdx].pt
        x2, y2 = kp2[matches[ind].trainIdx].pt
        
        _x, _y = x_prime(x1, y1, h), y_prime(x1, y1, h)

        distance = math.sqrt(math.pow((x2 - _x), 2) + math.pow((y2 - _y), 2))
        if distance < threshold:
            inliers += 1
    
    return inliers

In [287]:
def ransac(matches, kp1, kp2, iterations=100, threshold=5.0):
    """
    :param matches: total list of matches between two images
    :param kp1: keypoints from image 1
    :param kp2: keypoints from image 2
    :param iterations: number of times to run homography estimation on subset of matches
    :param threshold: distance threshold for considering a point as an inlier instead of an outlier
    :return: best homography matrix computed from the estimation process
    """
    
    best_inliers = 0
    best_homography = None
    best_acc = None
    
    for _ in range(iterations):
        # Randomly select 4 matches
        random_matches = np.random.choice(matches, 4, replace=False)
        
        # compute homography
        h = get_homography_matrix(kp1, kp2, random_matches)
        # compute error rate for this homography
        error = check_ransac_error(kp1, kp2, matches, h, threshold)
        if error > best_inliers:
            best_acc = error / len(matches)
            best_homography = h
            best_inliers = error
    
    print("best accuracy rate", best_acc)
    print("# of inliers:", best_inliers, "/", len(matches))
    
    return best_homography

In [288]:
def stitch_images(image1_path, image2_path):
    # Load the images
    img1 = cv2.imread(image1_path, cv2.COLOR_BGR2GRAY)
    img2 = cv2.imread(image2_path, cv2.COLOR_BGR2GRAY)
    
    # Initialize SIFT
    sift = cv2.SIFT_create()
    
    # Detect keypoints and descriptors
    kp1, desc1 = sift.detectAndCompute(img1, None)
    kp2, desc2 = sift.detectAndCompute(img2, None)
    
    ''' 
    using FLANN over KNN to find matches because it is faster at the cost cost of accuracy
    in comparison (as KNN is exhaustive). This is because image stitching does not need 100% accuracy
    and the tradeoff for computation speed is better for scalability. Ex: multiple 4k+ images
    '''
    
    # The index for KDTREE in FLANN
    FLANN_INDEX_KDTREE = 1
    # Initialize index params
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    # Initialize search params
    search_params = dict(checks=50)
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    # look for matches going from the image to the right -> left (not entirely sure why but the results is a lot more accurate)
    matches = flann.knnMatch(desc2, desc1, k=2)

    # Apply ratio test to filter good matches
    threshold=0.7
    good_matches = []
    for m, n in matches:
        if m.distance < threshold * n.distance:
            good_matches.append(m)

    # uncomment if you want to see matches visualized on the two images
    # draw_matches(img1, img2, kp2, kp1, good_matches)
    
    # Find Homography using RANSAC
    h = ransac(good_matches, kp2, kp1, 1000, 5.0)
    
    # Warp Image to so that you can stitch images together accurately
    panorama = cv2.warpPerspective(img2, h, ((img2.shape[1] + img1.shape[1]), img1.shape[0]))
    
    # Attach the first image on top of the warped image so that they build a nice panorama
    panorama[0:img1.shape[0], 0:img1.shape[1]] = img1

    return panorama


In [289]:
img1_path = 'landscape_1.jpg'
img2_path = 'landscape_2.jpg'
stitched_image = stitch_images(img1_path, img2_path)
cv2.imshow('Stitched Image', stitched_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

best accuracy rate 0.9555555555555556
# of inliers: 215 / 225
